In [11]:
import pandas as pd
from .. import utils
import plotly.graph_objects as go
from .. import instrument

In [12]:
pair = "GBP_JPY"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [13]:
df = pd.read_pickle(utils.get_hist_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [14]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True,inplace=True)

In [15]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
#compare diff and diff prev to see if there is a cross
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [16]:
#apply this function to all the rows of the data frame
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis = 1)

In [17]:
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [18]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [19]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
59,2022-01-26T05:00:00.000000000Z,153.907,153.907,153.812,153.858,153.782250,153.758672,0.023578,-0.012375,1,92.8,92.8
163,2022-02-01T13:00:00.000000000Z,154.592,154.810,154.541,154.786,154.727250,154.728297,-0.001047,0.006031,-1,25.5,-25.5
165,2022-02-01T15:00:00.000000000Z,154.910,155.204,154.882,155.041,154.752938,154.747281,0.005656,-0.001078,1,19.3,19.3
256,2022-02-07T10:00:00.000000000Z,155.466,155.570,155.140,155.234,155.868938,155.869797,-0.000859,0.049422,-1,112.4,-112.4
279,2022-02-08T09:00:00.000000000Z,156.222,156.433,156.128,156.358,156.003000,155.988734,0.014266,-0.027984,1,0.4,0.4


In [20]:
#16 MA and 64 MA performance; buy when 16 cross 64, sell when 64 cross 16
df_trades["GAIN"].sum()

1035.8000000000118

In [21]:
df_plot = df_ma.iloc[50:300].copy()
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df_plot.time, open = df_plot.mid_o, high = df_plot.mid_h, low = df_plot.mid_l, close = df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#23A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[col], line = dict(width = 2), line_shape='spline', name=col))
fig.update_layout(width=1000,height=400, font = dict(size=10, color = "#e1e1e1"), paper_bgcolor="#1e1e1e", plot_bgcolor="#1e1e1e")
fig.update_xaxes(gridcolor = "#1f292f", showgrid=True, fixedrange=True, rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor= "#1f292f", showgrid=True)
fig.show()